<a href="https://colab.research.google.com/github/skytecat/CreditScoring/blob/main/credit_scoring.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Загрузка библиотек

In [ ]:
# Установка необходимых библиотек
!pip install catboost
!pip install optuna

# Импорт библиотек
import numpy as np
import pandas as pd
from catboost import CatBoostClassifier
from sklearn.model_selection import StratifiedKFold, train_test_split
from sklearn.metrics import roc_auc_score
from sklearn.linear_model import LogisticRegression
from sklearn.preprocessing import StandardScaler
import optuna

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 MB 9.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 395.9/395.9 kB 14.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 247.0/247.0 kB 25.2 MB/s eta 0:00:00


## Загрузка данных

In [ ]:
train_full = pd.read_csv("/content/drive/MyDrive/SHIFT_ML_2025_COMPETITION/shift_ml_2025_train.csv")
test_final = pd.read_csv("/content/drive/MyDrive/SHIFT_ML_2025_COMPETITION/shift_ml_2025_test.csv")

/tmp/ipython-input-195565322.py:1: DtypeWarning: Columns (49) have mixed types. Specify dtype option on import or set low_memory=False.
  train_full = pd.read_csv("/content/drive/MyDrive/SHIFT_ML_2025_COMPETITION/shift_ml_2025_train.csv")


## Первичный анализ

In [ ]:
print(train_full.shape)
print(test_final.shape)

(1210779, 109)
(134531, 108)


In [ ]:
train_full.head(5)

,id,сумма_займа,срок_займа,процентная_ставка,аннуитет,рейтинг,допрейтинг,профессия_заемщика,стаж,владение_жильем,...,процент_счетов_прев_75_лимита,кол-во_публ_банкротств,кол-во_залогов,кредитный_лимит,кредитный_баланс_без_ипотеки,лимит_по_картам,лимит_по_аннуитетным_счетам,кредитный_баланс_по_возоб_счетам,особая_ситуация,тип_предоставления_кредита
0,68355089,1235000.0,3 года,11.99,41014.0,В,В1,инженер,10+ лет,ИПОТЕКА,...,7.7,0.0,0.0,15700850.0,1973750.0,3965000.0,1233350.0,NaN,Нет,Наличные
1,68341763,1000000.0,5 лет,10.78,21633.0,Б,Б4,водитель грузовика,10+ лет,ИПОТЕКА,...,50.0,0.0,0.0,10920900.0,934800.0,310000.0,743850.0,NaN,Нет,Наличные
2,68426831,597500.0,3 года,13.44,20259.0,В,В3,ветеринарный техник,4 года,АРЕНДА,...,100.0,0.0,0.0,845000.0,639900.0,470000.0,200000.0,NaN,Нет,Наличные
3,68476668,1000000.0,3 года,9.17,31879.0,Б,Б2,вице-президент операций по набору персонала,10+ лет,ИПОТЕКА,...,100.0,0.0,0.0,19442600.0,5838100.0,1575000.0,2322600.0,NaN,Нет,Наличные
4,67275481,1000000.0,3 года,8.49,31563.0,Б,Б1,дорожному водителю,10+ лет,ИПОТЕКА,...,0.0,0.0,0.0,9669500.0,1396850.0,725000.0,1807200.0,NaN,Нет,Наличные


In [ ]:
print("Распределение целевой переменной")
round(train_full['итоговый_статус_займа'].value_counts(normalize=True)*100, 2)

Распределение целевой переменной


,proportion
итоговый_статус_займа,
0,80.04
1,19.96


In [ ]:
# Столбцы в train
train_full.columns.to_list()

['id',
 'сумма_займа',
 'срок_займа',
 'процентная_ставка',
 'аннуитет',
 'рейтинг',
 'допрейтинг',
 'профессия_заемщика',
 'стаж',
 'владение_жильем',
 'годовой_доход',
 'подтвержден_ли_доход',
 'итоговый_статус_займа',
 'рейтинг_OCR',
 'платежный_график',
 'коэфф_акционных_зачислений_в_остатке',
 'цель_займа',
 'кол_во_субсидий_за_год',
 'индекс_проживания',
 'регион',
 'пдн',
 'пос_стоп_фактор',
 'юридический_статус',
 'кол-во_просрочек_за_два_года',
 'коэфф_невыплаченного_сумм_остатка',
 'коэфф_исходного_платежа',
 'дата_первого_займа',
 'нижний_порог_рейтинга_заемщика',
 'верхний_порог_рейтинга_заемщика',
 'кол-во_заявок_за_полгода',
 'кол-во_месяцев_с_последней_просрочки',
 'кол-во_месяцев_с_последнего_займа',
 'кол-во_открытых_счетов',
 'кол-во_записей_в_общедоступных_документах',
 'оборотный_баланс',
 'коэфф_загрузки_возобновляемого_счета',
 'кол-во_счетов',
 'первоначальный_статус_займа',
 'непогашенная_сумма_из_тела_займов',
 'сумма_выплат_по_просрочкам',
 'пени_за_дефолт',
 

In [ ]:
# Проверка на пересечение ID в тестовой и тренировочной выборке
overlap = len(set(train_full['id']) & set(test_final['id']))
print(f"Количество совпадающих ID: {overlap}")

Количество совпадающих ID: 0


Вывод: Количество совпадающих id между обучающей и тестовой выборками - 0. Это означает, что выборки независимы по идентификаторам заемщиков.


**Признак id необходимо удалить** из признаков, используемых для обучения модели, т.к. он не несет никакой информации о характеристиках заемщика, его поведении или рисках. В тестовой выборке будут совершенно новые id. Модель, обученная на старых id, не сможет использовать этот признак для предсказания. Это может привести к переобучению модели


Признак **пени_за_дефолт** вызывает подозрение на утечку данных (**data leakage**).


Если этот признак становится известен только после наступления события "дефолт", то использовать его для предсказания этого события - это нарушение причинно-следственной связи. Модель не должна знать "будущее"

In [ ]:
# Проверим уникальные значения в тестовой выборке
print("Уникальные значения 'пени_за_дефолт' в тесте:")
print(test_final['пени_за_дефолт'].unique())

Уникальные значения 'пени_за_дефолт' в тесте:
[nan]


Признак **пени_за_дефолт** в тестовой выборке не заполнен, что подтверждает его непригодность для использования в предсказательной модели

Посмотрим есть ли еще полностью пустые столбцы в тестовой выборке

In [ ]:
all_null_columns_test = test_final.columns[test_final.isna().all()].tolist()
all_null_columns_test

['пени_за_дефолт', 'дата_следующей_выплаты']

Удаление ненужных столбцов и разделение данных на признаки и целевую переменную

In [ ]:
X = train_full.copy()
y = train_full['итоговый_статус_займа']

X = X.drop(['пени_за_дефолт', 'id', 'дата_следующей_выплаты', 'итоговый_статус_займа'], axis=1)

In [ ]:
# Анализ типов данных признаков
print(X.dtypes.value_counts())

float64    83
object     18
int64       4
Name: count, dtype: int64


**Выбор модели:**

У нас есть 18 категориальных признаков. Обработка таких признаков требует особого подхода (например, One-Hot Encoding, Label Encoding). Многие алгоритмы (например, логистическая регрессия, SVM) не могут напрямую работать с текстовыми категориями.


**CatBoost** же разработан специально для эффективной и автоматической обработки категориальных данных. Его можно напрямую обучить на признаках типа object без необходимости предварительного ручного кодирования. Также он хорошо справляется с пропущенными значениями и не требует их обязательного заполнения средними/медианами.

Вывод: Наличие большого количества категориальных признаков (object) делает CatBoost идеальным кандидатом для этой задачи, обеспечивая при этом высокое качество, удобство использования и минимальную необходимость в ручной предобработке категорий.

## Baseline

In [ ]:
# Выберем только числовые признаки, чтобы избежать сложностей с кодированием категориальных переменных на этом этапе
num_features = X.select_dtypes(include=['int64', 'float64']).columns.tolist()
X_num = X[num_features]

# Заполним пропуски медианой
X_num = X_num.fillna(X_num.median())

# Используем только часть данных для ускорения обучения
sample_size = min(100_000, len(X_num))
idx_sample = np.random.RandomState(42).choice(X_num.index, size=sample_size, replace=False)
X_sample = X_num.loc[idx_sample]
y_sample = y.loc[idx_sample]

# Разделение на train и validation с учетом дисбаланса
X_train, X_val, y_train, y_val = train_test_split(
    X_sample, y_sample,
    test_size=0.2,
    stratify=y_sample,
    random_state=42
)

# Нормализация
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_val_scaled = scaler.transform(X_val)

# Обучение модели
model = LogisticRegression(
    C=1.0,
    solver='liblinear',
    max_iter=1000,
    random_state=42
)

model.fit(X_train_scaled, y_train)

# Оценка модели
val_pred = model.predict_proba(X_val_scaled)[:, 1]
auc_score = roc_auc_score(y_val, val_pred)
print(f"Logistic Regression AUC: {auc_score:.4f}")

Logistic Regression AUC: 0.7284


## Feature Engineering

Создаем **новые признаки** на основе существующих данных, чтобы предоставить модели дополнительную информацию и потенциально улучшить её предсказательную способность

In [ ]:
rating_rate = train_full.groupby('рейтинг')['итоговый_статус_займа'].mean()
X['платеж_на_доход'] = X['аннуитет'] / (X['годовой_доход'] / 12 + 1)
X['займ_на_доход'] = X['сумма_займа'] / (X['годовой_доход'] + 1)
X['отношение_просрочек'] = X['сумма_выплат_по_просрочкам'] / (X['сумма_займа'] + 1)
X['дефолт_по_рейтингу'] = X['рейтинг'].map(rating_rate).fillna(0)
X['доля_карт_без_нарушений'] = X['кол-во_карт_без_нарушений'] / (X['кол-во_карт'] + 1)
X['сумма_на_счетах_к_долгу'] = X['общая_сумма_на_счетах'] / (X['непогашенная_сумма_из_тела_займов'] + 1)
X['сумма_на_счетах_к_аннуитету'] = X['общая_сумма_на_счетах'] / (X['аннуитет'] + 1)
X['просрочка_к_доходу'] = X['сумма_выплат_по_просрочкам'] / (X['годовой_доход'] + 1)
X['баланс_к_аннуитету'] = X['оборотный_баланс'] / (X['аннуитет'] + 1)

**Созданные признаки:**

**платеж_на_доход:**
Отношение ежемесячного аннуитетного платежа к среднемесячному доходу заемщика. Показывает, какую долю своего дохода заемщик тратит на обслуживание этого займа.

**займ_на_доход**:
Отношение общей суммы займа к годовому доходу. Показывает, во сколько раз займ превышает годовой доход.

**отношение_просрочек:**
Доля средств, ушедших на погашение просрочек, от общей суммы займа.

**дефолт_по_рейтингу:**
Средняя частота дефолта для каждой категории признака рейтинг. Рассчитывается как среднее значение итоговый_статус_займа для каждого уникального рейтинга в обучающей выборке.

**доля_карт_без_нарушений:**
Доля карт заемщика, по которым отсутствуют нарушения.

**сумма_на_счетах_к\_долгу:**
Отношение доступных средств на счетах к оставшемуся основному долгу по займам.

**сумма_на_счетах_к\_аннуитету:**
Отношение доступных средств на счетах к ежемесячному аннуитетному платежу.

**просрочка_к\_доходу:**
Отношение общей суммы выплат по просрочкам к годовому доходу заемщика. Показывает, насколько просрочки соотносятся с годовым доходом клиента.

**баланс_к\_аннуитету:**
Отношение оборотного баланса к ежемесячному аннуитетному платежу. Отражает, во сколько раз баланс превышает обязательный ежемесячный платёж.

## Обучение модели и кросс-валидация

На этом этапе мы обучаем модель **CatBoostClassifier** — используя 5-фолдную стратифицированную кросс-валидацию для надежной оценки качества

In [ ]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
models = []
oof_preds = np.zeros(len(X))
cat_features = X.select_dtypes(include='object').columns.tolist()

# Заполняем пропуски в категориальных признаках
for col in cat_features:
    if col in X.columns:
        X[col] = X[col].fillna('nan').astype(str)

for train_idx, val_idx in kf.split(X, y):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    model = CatBoostClassifier(
        iterations=500,
        learning_rate=0.1,
        depth=8,
        l2_leaf_reg=3.0,
        random_strength=0.1,
        eval_metric='AUC',
        early_stopping_rounds=50,
        verbose=200,
        random_state=42,
        cat_features=cat_features,
        task_type='GPU'
    )

    model.fit(X_train, y_train, eval_set=(X_val, y_val), use_best_model=True)
    val_pred = model.predict_proba(X_val)[:, 1]
    oof_preds[val_idx] = val_pred

    models.append(model)

print(f"Validation OOF AUC: {roc_auc_score(y, oof_preds):.4f}")

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.7307337	best: 0.7307337 (0)	total: 867ms	remaining: 7m 12s
200:	test: 0.7604783	best: 0.7604783 (200)	total: 52.9s	remaining: 1m 18s
400:	test: 0.7644164	best: 0.7644164 (400)	total: 1m 45s	remaining: 26s
499:	test: 0.7653678	best: 0.7653678 (498)	total: 2m 10s	remaining: 0us
bestTest = 0.765367806
bestIteration = 498
Shrink model to first 499 iterations.


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.7303998	best: 0.7303998 (0)	total: 265ms	remaining: 2m 12s
200:	test: 0.7614335	best: 0.7614335 (200)	total: 53.1s	remaining: 1m 18s
400:	test: 0.7643579	best: 0.7643579 (400)	total: 1m 45s	remaining: 26s
499:	test: 0.7657062	best: 0.7657110 (497)	total: 2m 11s	remaining: 0us
bestTest = 0.7657110095
bestIteration = 497
Shrink model to first 498 iterations.


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.7300332	best: 0.7300332 (0)	total: 261ms	remaining: 2m 10s
200:	test: 0.7632975	best: 0.7632975 (200)	total: 53.2s	remaining: 1m 19s
400:	test: 0.7664758	best: 0.7664787 (398)	total: 2m	remaining: 29.7s
499:	test: 0.7670150	best: 0.7670150 (499)	total: 2m 26s	remaining: 0us
bestTest = 0.7670149803
bestIteration = 499


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.7289789	best: 0.7289789 (0)	total: 259ms	remaining: 2m 9s
200:	test: 0.7615907	best: 0.7615907 (200)	total: 53.9s	remaining: 1m 20s
400:	test: 0.7653132	best: 0.7653149 (399)	total: 1m 47s	remaining: 26.4s
499:	test: 0.7658879	best: 0.7658881 (498)	total: 2m 13s	remaining: 0us
bestTest = 0.7658880949
bestIteration = 498
Shrink model to first 499 iterations.


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.7287782	best: 0.7287782 (0)	total: 253ms	remaining: 2m 6s
200:	test: 0.7629196	best: 0.7629196 (200)	total: 52.6s	remaining: 1m 18s
400:	test: 0.7658138	best: 0.7658138 (400)	total: 1m 45s	remaining: 25.9s
499:	test: 0.7663519	best: 0.7663572 (498)	total: 2m 11s	remaining: 0us
bestTest = 0.7663572133
bestIteration = 498
Shrink model to first 499 iterations.
Validation OOF AUC: 0.7661


После обучения модели мы можем проанализировать, насколько каждый признак **"важен"** для принятия решений моделью. Это позволяет понять, какие переменные наиболее влияют на предсказание риска дефолта

In [ ]:
# Получение важности признаков от модели CatBoost
importances = model.feature_importances_
feature_names = X.columns

# Создание DataFrame для удобного анализа
importance_df = pd.DataFrame({
    'feature': feature_names,
    'importance': importances
}).sort_values('importance', ascending=False)


Признаки с высокой важностью (топ 20)

In [ ]:
importance_df.head(20)

,feature,importance
5,допрейтинг,11.359204
112,просрочка_к_доходу,7.934688
1,срок_займа,5.123795
37,сумма_выплат_по_просрочкам,3.761608
4,рейтинг,3.713082
6,профессия_заемщика,3.428830
18,пдн,3.333601
2,процентная_ставка,3.073628
65,кол-во_открытых_счетов_за_2_года,2.857542
17,регион,2.551699


Удаляем признаки с низкой важностью

In [ ]:
threshold = 0.01

features_to_keep = importance_df[importance_df['importance'] >= threshold]['feature'].tolist()

In [ ]:
X = X[features_to_keep]

In [ ]:
kf = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
models = []
oof_preds = np.zeros(len(X))
cat_features = X.select_dtypes(include='object').columns.tolist()

# Заполняем пропуски в категориальных признаках
for col in cat_features:
    if col in X.columns:
        X[col] = X[col].fillna('nan').astype(str)

for train_idx, val_idx in kf.split(X, y):
    X_train, X_val = X.iloc[train_idx], X.iloc[val_idx]
    y_train, y_val = y.iloc[train_idx], y.iloc[val_idx]

    model = CatBoostClassifier(
        iterations=500,
        learning_rate=0.1,
        depth=8,
        l2_leaf_reg=3.0,
        random_strength=0.1,
        eval_metric='AUC',
        early_stopping_rounds=50,
        verbose=200,
        random_state=42,
        cat_features=cat_features,
        task_type='GPU'
    )

    model.fit(X_train, y_train, eval_set=(X_val, y_val), use_best_model=True)
    val_pred = model.predict_proba(X_val)[:, 1]
    oof_preds[val_idx] = val_pred

    models.append(model)

print(f"Validation OOF AUC: {roc_auc_score(y, oof_preds):.4f}")

Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.7300094	best: 0.7300094 (0)	total: 287ms	remaining: 2m 23s
200:	test: 0.7608136	best: 0.7608136 (200)	total: 56.1s	remaining: 1m 23s
400:	test: 0.7647470	best: 0.7647470 (400)	total: 1m 49s	remaining: 27s
499:	test: 0.7655801	best: 0.7655801 (499)	total: 2m 15s	remaining: 0us
bestTest = 0.7655801475
bestIteration = 499


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.7303998	best: 0.7303998 (0)	total: 392ms	remaining: 3m 15s
200:	test: 0.7612983	best: 0.7612983 (200)	total: 53.5s	remaining: 1m 19s
400:	test: 0.7643830	best: 0.7643830 (400)	total: 1m 45s	remaining: 26s
499:	test: 0.7655399	best: 0.7655399 (499)	total: 2m 11s	remaining: 0us
bestTest = 0.7655398846
bestIteration = 499


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.7300300	best: 0.7300300 (0)	total: 281ms	remaining: 2m 19s
200:	test: 0.7632817	best: 0.7632817 (200)	total: 53.9s	remaining: 1m 20s
400:	test: 0.7666446	best: 0.7666462 (399)	total: 1m 46s	remaining: 26.3s
499:	test: 0.7672121	best: 0.7672121 (499)	total: 2m 12s	remaining: 0us
bestTest = 0.7672121227
bestIteration = 499


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.7277080	best: 0.7277080 (0)	total: 456ms	remaining: 3m 47s
200:	test: 0.7611558	best: 0.7611558 (200)	total: 53.6s	remaining: 1m 19s
400:	test: 0.7645883	best: 0.7645883 (400)	total: 1m 46s	remaining: 26.2s
499:	test: 0.7651131	best: 0.7651168 (497)	total: 2m 12s	remaining: 0us
bestTest = 0.7651168406
bestIteration = 497
Shrink model to first 498 iterations.


Default metric period is 5 because AUC is/are not implemented for GPU


0:	test: 0.7257990	best: 0.7257990 (0)	total: 308ms	remaining: 2m 33s
200:	test: 0.7624089	best: 0.7624089 (200)	total: 51.8s	remaining: 1m 17s
400:	test: 0.7656496	best: 0.7656496 (400)	total: 1m 44s	remaining: 25.7s
499:	test: 0.7665180	best: 0.7665180 (499)	total: 2m 9s	remaining: 0us
bestTest = 0.7665179968
bestIteration = 499
Validation OOF AUC: 0.7660
